In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForQuestionAnswering
from helper.DocRetriever import DocRetriever
from helper.DocReader import DocReader


I0411 23:42:45.265548 4468841792 file_utils.py:41] PyTorch version 1.4.0 available.
/Users/madhuhegde/.pyenv/versions/miniconda3-latest/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/madhuhegde/.pyenv/versions/miniconda3-latest/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/madhuhegde/.pyenv/versions/miniconda3-latest/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be und

In [2]:
# Read corpus
with open('./input/covid_corpus.csv', 'rb') as f:
    frame = pd.read_csv(f)

#Document retriver based on BM250    
retriever = DocRetriever(top_n=10)

#Vectorize BM250 using the corpus
retriever.fit_retriever(frame)

#Reader based on Hugging Face BertForQuestionAnswering Transformer
#reader = DocReader('./input/model/')
reader = DocReader('bert-large-uncased-whole-word-masking-finetuned-squad')

I0411 23:44:01.178651 4468841792 tokenization_utils.py:504] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-whole-word-masking-finetuned-squad-vocab.txt from cache at /Users/madhuhegde/.cache/torch/transformers/425869b134694b14f3fd06adbee0d84e62abc76097792a5c6f94524239e0e8b4.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I0411 23:44:01.623034 4468841792 configuration_utils.py:275] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-whole-word-masking-finetuned-squad-config.json from cache at /Users/madhuhegde/.cache/torch/transformers/00bf8b18639562831ff9d22825e646f00f34a6976c6ed0ed1c4b35d334bb7a51.84dc10193e066c2b505f965e0ba0751ef60dba0afccfad2e23acd97eb7feec5e
I0411 23:44:01.624878 4468841792 configuration_utils.py:311] Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  

In [3]:
# Find top_n documents based on BM250 for the query 
query = "what is covid-19"
doc_scores = retriever.compute_scores(query)

#Select top_n documents
index = [score[0] for score in doc_scores]
text = frame.loc[index]

#predict n_best answers using BERT
answers = reader.predict(df=text, query=query, n_best=5)

#Select best answer based on logitic values
b_answer = reader.best_answer(answers)

print('query: {}\n'.format(query))
print('answer: {}\n'.format(b_answer['answer']))
print('title: {}\n'.format(text['title'][b_answer['index']]))
print('paragraph: {}\n'.format(b_answer['sentence']))


query: what is covid-19

answer: coronavirus disease 2019

title: Clinical findings in a group of patients infected with the 2019 novel coronavirus (SARS-Cov-2) outside of Wuhan, China: retrospective case series

paragraph: 9 coronavirus disease 2019 (covid-19) soon drew global attention because of the rapidly increasing numbers of new cases.

